In [16]:
import numpy as np
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

mnist = fetch_openml("mnist_784", parser = 'auto')

X = mnist.data.values # X를 배열 형태로 지정
X = (X > 128).astype(int)
    
y = mnist.target.values.astype(int)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

class BN:
    def __init__(self):
        self.prob_c = []
        self.prob_p = []

    
    def fit(self, X, y):
        #class probability
        for c in np.unique(y):
            num_c = np.sum(y == c)
            prob_c = (num_c + 1) / len(X)
            self.prob_c.append(np.log(prob_c))

            num1 = X[y == c].sum(axis=0) + 1
            prob_p = num1 / num_c
            self.prob_p.append(np.log(prob_p))

    def predict(self, X):
        y_pred = []

        for x in X:
            class_scores = []
        
            for c, c_prob in zip(self.prob_c, self.prob_p):
                p_prob = 0
                for i, pixel_value in enumerate(x):
                    if pixel_value == 1:
                        p_prob += c_prob[i]
                    else:
                        p_prob += np.log(1 - np.exp(c_prob[i]))
                class_scores.append(c + p_prob)
        
            predicted_class = np.argmax(class_scores)
            y_pred.append(predicted_class)

        return y_pred



model = BN()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: {:.2f}%".format(accuracy * 100))

Accuracy: 83.76%
